In [75]:
import pandas as pd
import numpy as np
import joblib
from scipy.sparse import hstack, csr_matrix
import re


In [76]:
df_test = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\test_processed.csv")


In [77]:
for col in ['item_name', 'bullet_points', 'product_description']:
    df_test[col] = df_test[col].fillna('')

In [78]:
df_test['text_all'] = df_test['item_name'] + ' ' + df_test['bullet_points'] + ' ' + df_test['product_description']


In [79]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()
df_test['text_all'] = df_test['text_all'].apply(clean_text)

In [80]:
tfidf = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\tfidf_vectorizer.pkl")
ohe = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\onehot_encoder.pkl")
xgb_model = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\xgb_model.pkl")


In [81]:
text_features = tfidf.transform(df_test['text_all'])
cat_features = ohe.transform(df_test[['unit_standard']].fillna('MISSING'))

In [82]:
numeric_features = ["quantity_numeric", "price_per_unit", "value"]
for col in numeric_features:
    if col not in df_test.columns:
        df_test[col] = 0
X_num = df_test[numeric_features].fillna(0)


In [83]:
X_test_final = hstack([csr_matrix(X_num.values), cat_features, text_features])

In [84]:
expected_shape = xgb_model.get_booster().num_features()
if X_test_final.shape[1] != expected_shape:
    diff = expected_shape - X_test_final.shape[1]
    if diff > 0:
        X_test_final = hstack([X_test_final, csr_matrix((X_test_final.shape[0], diff))])
    elif diff < 0:
        X_test_final = X_test_final[:, :expected_shape]

print("✅ Final test feature matrix shape:", X_test_final.shape)

✅ Final test feature matrix shape: (75000, 5016)


In [85]:
y_pred_log = xgb_model.predict(X_test_final)
y_pred = np.expm1(y_pred_log) 

In [86]:
if 'sample_id' not in df_test.columns:
    df_test['sample_id'] = range(1, len(df_test) + 1)

In [87]:
submission_df = pd.DataFrame({
    'sample_id': df_test['sample_id'],
    'price': y_pred
})

submission_df.to_csv("submission.csv", index=False)
print("✅ Submission CSV created successfully!")

✅ Submission CSV created successfully!
